In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(100))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [13]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('selu')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 27s 79ms/step - loss: 4.5455 - accuracy: 0.0384 - top3_acc: 0.0953 - top_k_categorical_accuracy: 0.1461 - val_loss: 3.8670 - val_accuracy: 0.1265 - val_top3_acc: 0.2566 - val_top_k_categorical_accuracy: 0.3428
Epoch 2/50
313/313 [==============================] - 25s 79ms/step - loss: 3.9152 - accuracy: 0.1042 - top3_acc: 0.2281 - top_k_categorical_accuracy: 0.3159 - val_loss: 3.6635 - val_accuracy: 0.1535 - val_top3_acc: 0.3046 - val_top_k_categorical_accuracy: 0.3970
Epoch 3/50
313/313 [==============================] - 25s 79ms/step - loss: 3.6931 - accuracy: 0.1446 - top3_acc: 0.2950 - top_k_categorical_accuracy: 0.3920 - val_loss: 3.5366 - val_accuracy: 0.1778 - val_top3_acc: 0.3385 - val_top_k_categorical_accuracy: 0.4366
Epoch 4/50
313/313 [==============================] - 24s 76ms/step - loss: 3.5639 - accuracy: 0.1684 - top3_acc: 0.3314 - top_k_categorical_accuracy: 0.4352 - val_loss: 3.3979 - val_accuracy: 0.2046 - val_to

In [14]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [4.293421268463135, 3.8543620109558105, 3.6633479595184326, 3.5303571224212646, 3.4082984924316406, 3.3054206371307373, 3.215646982192993, 3.131589651107788, 3.050485849380493, 2.981128215789795, 2.910179376602173, 2.847477436065674, 2.7819998264312744, 2.72220516204834, 2.6624789237976074, 2.6074955463409424, 2.545815944671631, 2.4906086921691895, 2.4358222484588623, 2.3848752975463867, 2.3327488899230957, 2.279548406600952, 2.223081350326538, 2.1843349933624268, 2.128784656524658, 2.075688600540161, 2.01882266998291, 1.979316234588623, 1.9226524829864502, 1.8721306324005127, 1.823724389076233, 1.7781299352645874, 1.727211356163025, 1.6892601251602173, 1.631411075592041, 1.5878374576568604, 1.5374473333358765, 1.4828646183013916, 1.4456076622009277, 1.4004451036453247, 1.3490406274795532, 1.3034908771514893, 1.2582390308380127, 1.2226592302322388, 1.1834745407104492, 1.1373447179794312, 1.0954023599624634, 1.056567907333374, 1.0113214254379272, 0.9810063242912292], 'accuracy'